In [34]:
import os
from datetime import datetime, timedelta  # For working with dates

import pandas as pd  # For working with DataFrames
from dotenv import load_dotenv
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
from sqlalchemy import create_engine  # For crea

##################### LOADING IMPORTANT DATA ######################
# Load environment variables from the .env file
env_file_path = 'D:/Projects/.env'
load_dotenv(env_file_path)
# Giving output file name
output_file_path = 'HOURLY.xlsx'
# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = 'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

##################### ACCESS ENV VARIABLES ######################
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_SERGELI")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

##################### PROCEDURE NAME ######################
procedure_name = 'zAdmReportDFS_short'  # THIS IS HOURLY DATA GATHERING

##################### DATE - JANUARY ######################
CURRENT_MONTH = 1
CURRENT_YEAR = 2024
today_date = datetime.now().strftime('%d/%m/%Y')
tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%d/%m/%Y')
##################### CONNECTION STRING AND SQL QUERY ######################
# Construct the connection string
conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)

sql_query = f"""
    DECLARE @DateBegin DATE = ?;
    DECLARE @DateEnd DATE = ?;

    EXEC {procedure_name}
        @DateBegin = @DateBegin,
        @DateEnd = @DateBegin;
"""

#####################  EXECUTION  ######################
df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))

In [35]:
df.columns = [
    'DocumentType', 'Invoice Number', 'Goodid', 'Good', 'Manufacturer',
    'inn', 'ClientName', 'SalesManager', 'ClientMan', 'PaymentTerm',
    'BasePrice', 'SellingPrice', 'Quantity', 'DateEntered', 'BaseAmount', 'TotalAmount'
]

In [36]:
##################### BASIC FILTER ######################
df = df[df['DocumentType'].isin(['Оптовая реализация', 'Финансовая скидка'])]

In [37]:

df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientMan', right_on='ClientMan', how='left')

df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')


In [38]:
df['inn_temp'] = pd.to_numeric(df['inn'], errors='coerce')
types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
df = pd.merge(df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp', how='left')
df['TYPE'].fillna('ROZ', inplace=True)
df.loc[df['TYPE'] == 'ROZ', 'RegionType'] = df['Region']

df['OXVAT'] = df['inn'].map(df['inn'].value_counts())

In [39]:
categorical_columns = ['DocumentType', 'Good', 'Manufacturer', 'inn', 'ClientName', 'SalesManager', 'ClientMan',
                       'PaymentTerm', 'Region', 'RegionType', 'TYPE']

In [40]:
df[categorical_columns] = df[categorical_columns].astype('category')

In [41]:
# Assuming df is your DataFrame
columns_to_drop = [col for col in df.columns if col.endswith('_temp')]

# Drop the identified columns
df.drop(columns=columns_to_drop, inplace=True)
df.to_excel(output_file_path,)

# FORMATTING

In [42]:
# Load the existing workbook
workbook = load_workbook(output_file_path)

# Access the default sheet (assuming it's the only sheet in the workbook)
worksheet = workbook.active

# 1. Color first row (headers) with 4CB9E7 color code
header_style = NamedStyle(name='header_style', fill=PatternFill(
    start_color='4CB9E7', end_color='4CB9E7', fill_type='solid'))

for cell in worksheet[1]:
    cell.style = header_style

# 2. Autofit all columns
for column in worksheet.columns:
    max_length = 0
    column = [cell for cell in column]
    for cell in column:
        try:
            value = str(cell.value)
            if len(value) > max_length:
                max_length = len(value)
        except:
            pass
    adjusted_width = (max_length + 5)
    column_letter = get_column_letter(column[0].column)
    worksheet.column_dimensions[column_letter].width = adjusted_width

# 3. Format columns with thousands separators
number_format = NamedStyle(
    name='number_format', number_format='### ### ### ##0')

# Specify the columns to format based on float64 datatype
# Specify the columns to format based on float64 datatype
float64_columns = df.select_dtypes(include=['float64']).columns
for col in float64_columns:
    col_index = df.columns.get_loc(col) + 1  # 1-based index
    col_letter = get_column_letter(col_index)

    for cell in worksheet[col_letter][1:]:  # Start from the second row assuming the first row is headers
        try:
            formatted_value = "{:,.2f}".format(float(cell.value))
            cell.value = float(cell.value)
            cell.style = number_format
        except (ValueError, TypeError):
            # Handle cases where the cell value is not a valid number
            pass

# # Get the last column letter
# last_column_letter = get_column_letter(worksheet.max_column)
# 
# # Apply background color to all cells in the last column
# for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=worksheet.max_column,
#                                max_col=worksheet.max_column):
#     for cell in row:
#         cell.fill = PatternFill(start_color='A0D8B3', end_color='A0D8B3', fill_type='solid')

# Add borders to all cells with data
for row in worksheet.iter_rows(min_row=1, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column):
    for cell in row:
        cell.border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )

workbook.save(output_file_path)

In [43]:
def calculate_oxvat(filtered_d, region):
    """
    Calculate the count of unique clients based on the provided conditions.

    Parameters:
    - filtered_df (DataFrame): A DataFrame filtered to include only rows where 'TYPE' is 'ROZ'.
    - region (str): The specific region for which to calculate the count of unique clients.

    Returns:
    int: The count of unique clients for the given region.
    """
    return len(filtered_d[(filtered_d['RegionType'] == region) & (filtered_d['TYPE'] == 'ROZ')]['inn'].unique())


def calculate_fact_to(region, region_type, *aksiya_value):
    """
    Calculate the 'Факт ТО' for a specific region, product type, and aksiya value.

    Parameters:
    - region (str): The specific region for which to calculate 'Факт ТО'.
    - region_type (str): The specific product type for which to calculate 'Факт ТО'.
    - aksiya_value (str, optional): The specific value for 'aksiya' to be considered in the calculation.

    Returns:
    float: The calculated 'Факт ТО'.
    """
    filtered_region_df = filtered_df[(filtered_df['Region'] == region) & (filtered_df['TYPE'] == region_type)]

    if aksiya_value:
        filtered_region_df = filtered_region_df[filtered_region_df['Aksiya'] == aksiya_value[0]]
        return max(filtered_region_df['Kolich'].sum(), 0)

    return max(filtered_region_df['TotalAmount'].sum(), 0)


def calculate_percentage(fact_column, plan_column):
    """
    Calculate the percentage based on 'Факт' and 'План' columns.

    Parameters:
    - fact_column (Series): The Series representing the 'Факт' column.
    - plan_column (Series): The Series representing the 'План' column.

    Returns:
    Series: The calculated percentage column.
    """
    return (fact_column / plan_column).fillna(1) * 100


# Load data from hourly.xlsx
input_file_path = 'Hourly.xlsx'
plan_path = 'D:/Projects/plan.xlsx'
# Read plan data from plan.xlsx
plan_df = pd.read_excel(plan_path, sheet_name="ROZ")
df = pd.read_excel(input_file_path)

# Filter the DataFrame to include only rows where 'TYPE' is 'ROZ'
filtered_df = df[df['TYPE'] == 'ROZ']

# Define unique values for TYPE
unique_types = filtered_df['TYPE'].unique()

# Defined the specific Plan Columns i need
region_combinations = plan_df[['Region', 'TO', 'Rinomaks', 'Forsil', 'Тризим Таб', 'Тризим Кап', 'Энтеросгель']]

# Calculate 'Факт ТО' and '%' for each product type
for product_type in unique_types:
    fact_column = f'Fact ТО {product_type}'
    percent_column = f'% {product_type}'

    region_combinations[fact_column] = region_combinations['Region'].apply(
        lambda region: calculate_fact_to(region, product_type))

    region_combinations[percent_column] = calculate_percentage(region_combinations[fact_column],
                                                               region_combinations['TO'])

# Add 'OXVAT' column based on the provided formula
region_combinations['OXVAT'] = region_combinations['Region'].apply(lambda region: calculate_oxvat(filtered_df, region))

# Add Rinomaks columns
region_combinations['Fact Rinomaks'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Рино'))
region_combinations['% Rinomaks'] = calculate_percentage(region_combinations['Fact Rinomaks'],
                                                         region_combinations['Rinomaks'])
# FORSIL
region_combinations['Fact FORSIL'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Форсил'))
region_combinations['% FORSIL'] = calculate_percentage(region_combinations['Fact FORSIL'],
                                                       region_combinations['Forsil'])

region_combinations['Fact Тризим Таб'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Тризим №20'))
region_combinations['% Тризим Таб'] = calculate_percentage(region_combinations['Fact Тризим Таб'],
                                                           region_combinations['Тризим Таб'])

region_combinations['Fact Тризим Кап'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Тризим Кап'))
region_combinations['% Тризим Кап'] = calculate_percentage(region_combinations['Fact Тризим Кап'],
                                                           region_combinations['Тризим Кап'])

region_combinations['Fact Энтеросгель'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Энтеросгель'))
region_combinations['% Энтеросгель'] = calculate_percentage(region_combinations['Fact Энтеросгель'],
                                                            region_combinations['Энтеросгель'])
# COLUMN RENAME TO RUSSIAN LETTERS FROM ENGLISH
column_mapping = {
    'Region': 'Регион',
    'TO': 'TO',
    'Fact ТО ROZ': 'Факт ТО',
    '% ROZ': '% TO',
    'OXVAT': 'Охват',
    'Rinomaks': 'Риномакс',
    'Fact Rinomaks': 'Факт Риномакс',
    '% Rinomaks': '% Риномакс',
    'Forsil': 'Форсил',
    'Fact FORSIL': 'Факт Форсил',
    '% FORSIL': '% Форсил',
    'Тризим Таб': 'Тризим Таб',
    'Fact Тризим Таб': 'Факт Тризим Таб',
    '% Тризим Таб': '% Тризим Таб',
    'Тризим Кап': 'Тризим Кап',
    'Fact Тризим Кап': 'Факт Тризим Кап',
    '% Тризим Кап': '% Тризим Кап',
    'Энтеросгель': 'Энтеросгель',
    'Fact Энтеросгель': 'Факт Энтеросгель',
    '% Энтеросгель': '% Энтеросгель'
}

# Rename the columns using the mapping dictionary
region_combinations = region_combinations.rename(columns=column_mapping)
# Sorting to look it pretty
region_combinations.sort_values(by='% TO', ascending=False, inplace=True)

KeyError: 'Kolich'

In [ ]:
total_row= pd.Series({
    'Регион': 'TOTAL',
    'TO': region_combinations['TO'].sum(),
    'Факт ТО': region_combinations['Факт ТО'].sum(),
    '% TO': (region_combinations['Факт ТО'].sum() / region_combinations['TO'].sum()) * 100,
    'Охват': region_combinations['Охват'].sum(),
    'Риномакс': region_combinations['Риномакс'].sum(),
    'Факт Риномакс': region_combinations['Факт Риномакс'].sum(),
    '% Риномакс': (region_combinations['Факт Риномакс'].sum() / region_combinations['Риномакс'].sum()) * 100,
    'Форсил': region_combinations['Форсил'].sum(),
    'Факт Форсил': region_combinations['Факт Форсил'].sum(),
    '% Форсил': (region_combinations['Факт Форсил'].sum() / region_combinations['Форсил'].sum()) * 100,
    'Тризим Таб': region_combinations['Тризим Таб'].sum(),
    'Факт Тризим Таб': region_combinations['Факт Тризим Таб'].sum(),
    '% Тризим Таб': (region_combinations['Факт Тризим Таб'].sum() / region_combinations['Тризим Таб'].sum()) * 100,
    'Тризим Кап': region_combinations['Тризим Кап'].sum(),
    'Факт Тризим Кап': region_combinations['Факт Тризим Кап'].sum(),
    '% Тризим Кап': (region_combinations['Факт Тризим Кап'].sum() / region_combinations['Тризим Кап'].sum()) * 100,
    'Энтеросгель': region_combinations['Энтеросгель'].sum(),
    'Факт Энтеросгель': region_combinations['Факт Энтеросгель'].sum(),
    '% Энтеросгель': (region_combinations['Факт Энтеросгель'].sum() / region_combinations['Энтеросгель'].sum()) * 100
}, name='TOTAL')

region_combinations = pd.concat([region_combinations, total_row.to_frame().transpose()], ignore_index=True)
region_combinations=region_combinations[['Регион', 'TO', 'Факт ТО', '% TO', 'Охват', 'Риномакс', 'Факт Риномакс',
                     '% Риномакс', 'Форсил', 'Факт Форсил', '% Форсил', 'Тризим Таб', 'Факт Тризим Таб',
                     '% Тризим Таб', 'Тризим Кап', 'Факт Тризим Кап', '% Тризим Кап',
                     'Энтеросгель', 'Факт Энтеросгель', '% Энтеросгель']]

In [44]:
region_combinations.to_excel('Salom.xlsx',index=False)